# 顔検出課題 - Face Detection Assignment

## テーマ：公共スペースの混雑（人数）推定

この課題では、YOLOとMTCNNの2つの顔検出手法を使用して、公共スペース内の人数を推定します。

### 使用手法：
1. **YOLO (You Only Look Once)** - 高速リアルタイム検出
2. **MTCNN (Multi-task Cascaded Convolutional Networks)** - 高精度検出

### 分析内容：
- 各画像に写っている人数のカウント
- 検出が難しいケースの分析（小さい顔、横向き、暗い場所など）
- YOLOとMTCNNの比較

## 1. セットアップ - Setup

必要なライブラリをインストールします。

In [ ]:
# 必要なライブラリのインストール
!pip install ultralytics mtcnn opencv-python pillow matplotlib numpy

In [ ]:
# ライブラリのインポート
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
from mtcnn import MTCNN
from ultralytics import YOLO
import time

# 日本語フォント設定（matplotlib用）
plt.rcParams['font.sans-serif'] = ['MS Gothic', 'Yu Gothic', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("ライブラリのインポートが完了しました")

## 2. 画像の読み込み - Load Images

分析用のサンプル画像を読み込みます。

- **sample1_bright.png**: 明るい場所でのグループ写真
- **sample2_crowd.png**: 混雑した公共スペース（様々な角度の顔）
- **sample3_lowlight.png**: 暗い場所での写真

In [ ]:
# 画像ファイルのパス
image_paths = [
    'sample1_bright.png',
    'sample2_crowd.png',
    'sample3_lowlight.png'
]

# 画像の読み込み
images = []
for path in image_paths:
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img_rgb)

# 元画像の表示
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
titles = ['明るい場所（グループ）', '混雑した公共スペース', '暗い場所']

for idx, (img, title) in enumerate(zip(images, titles)):
    axes[idx].imshow(img)
    axes[idx].set_title(title, fontsize=12)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print(f"読み込んだ画像数: {len(images)}")

## 3. YOLO による顔検出

### YOLOの特徴：
**強み（役に立つ点）：**
- ⚡ **高速処理**: リアルタイム検出が可能で、動画解析にも使える
- 🎯 **シンプルな実装**: 使いやすく、導入が簡単
- 💪 **汎用性**: 様々なサイズの物体を検出できる

**弱み（限界を感じる点）：**
- 👤 **小さい顔の検出**: 遠くの小さな顔や低解像度の顔は検出しづらい
- 🔄 **重なり**: 複数の顔が重なっている場合、検出率が下がる
- 🌙 **暗い環境**: 低照度の環境では精度が落ちやすい

In [ ]:
# YOLOモデルの初期化（顔検出用のYOLOv8モデル）
# 注意: 初回実行時はモデルのダウンロードが行われます
print("YOLOモデルを読み込んでいます...")
yolo_model = YOLO('yolov8n.pt')  # nanoモデル（軽量版）を使用
print("YOLOモデルの読み込み完了")

In [ ]:
# YOLOで顔検出を実行
yolo_results = []
yolo_times = []

for idx, img in enumerate(images):
    print(f"\n画像 {idx+1} をYOLOで処理中...")
    
    # 処理時間の計測
    start_time = time.time()
    
    # YOLO検出（personクラスのみ）
    results = yolo_model(img, conf=0.3)  # 信頼度30%以上
    
    end_time = time.time()
    processing_time = end_time - start_time
    yolo_times.append(processing_time)
    
    # 人物（person）の検出結果を抽出
    detections = []
    for r in results:
        boxes = r.boxes
        for box in boxes:
            cls = int(box.cls[0])
            # クラス0は「person」
            if cls == 0:  
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                conf = float(box.conf[0])
                detections.append({
                    'box': [x1, y1, x2, y2],
                    'confidence': conf
                })
    
    yolo_results.append(detections)
    print(f"  検出人数: {len(detections)}人")
    print(f"  処理時間: {processing_time:.3f}秒")

In [ ]:
# YOLO検出結果の可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
titles = ['明るい場所（グループ）', '混雑した公共スペース', '暗い場所']

for idx, (img, detections, title) in enumerate(zip(images, yolo_results, titles)):
    axes[idx].imshow(img)
    
    # バウンディングボックスを描画
    for det in detections:
        x1, y1, x2, y2 = det['box']
        w = x2 - x1
        h = y2 - y1
        rect = Rectangle((x1, y1), w, h, 
                         linewidth=2, edgecolor='lime', facecolor='none')
        axes[idx].add_patch(rect)
        
        # 信頼度を表示
        conf_text = f"{det['confidence']:.2f}"
        axes[idx].text(x1, y1-5, conf_text, 
                      color='lime', fontsize=10, weight='bold',
                      bbox=dict(boxstyle='round', facecolor='black', alpha=0.5))
    
    axes[idx].set_title(f"{title}\nYOLO検出: {len(detections)}人", fontsize=12)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 4. MTCNN による顔検出

### MTCNNの特徴：
**強み（役に立つ点）：**
- 🎯 **高精度**: 小さい顔や部分的に隠れた顔も検出しやすい
- 👁️ **顔パーツ検出**: 目、鼻、口の位置も取得できる（顔認識に有用）
- 📐 **多様な角度**: 横向きや斜めの顔も比較的よく検出できる

**弱み（限界を感じる点）：**
- ⏱️ **処理速度**: YOLOに比べて処理が遅く、リアルタイム用途には不向き
- 💻 **計算コスト**: CPUでの処理が重い（GPUがあれば改善）
- 🌙 **極端な条件**: 極端に暗い場所やぼやけた画像では精度が落ちる

In [ ]:
# MTCNNモデルの初期化
print("MTCNNモデルを初期化しています...")
mtcnn_detector = MTCNN(min_face_size=20)  # 最小顔サイズ20px
print("MTCNNモデルの初期化完了")

In [ ]:
# MTCNNで顔検出を実行
mtcnn_results = []
mtcnn_times = []

for idx, img in enumerate(images):
    print(f"\n画像 {idx+1} をMTCNNで処理中...")
    
    # 処理時間の計測
    start_time = time.time()
    
    # MTCNN検出
    detections = mtcnn_detector.detect_faces(img)
    
    end_time = time.time()
    processing_time = end_time - start_time
    mtcnn_times.append(processing_time)
    
    mtcnn_results.append(detections)
    print(f"  検出人数: {len(detections)}人")
    print(f"  処理時間: {processing_time:.3f}秒")
    
    # 各検出の信頼度を表示
    if detections:
        confidences = [d['confidence'] for d in detections]
        print(f"  平均信頼度: {np.mean(confidences):.3f}")

In [ ]:
# MTCNN検出結果の可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
titles = ['明るい場所（グループ）', '混雑した公共スペース', '暗い場所']

for idx, (img, detections, title) in enumerate(zip(images, mtcnn_results, titles)):
    axes[idx].imshow(img)
    
    # バウンディングボックスと顔パーツを描画
    for det in detections:
        x, y, w, h = det['box']
        
        # バウンディングボックス
        rect = Rectangle((x, y), w, h, 
                         linewidth=2, edgecolor='cyan', facecolor='none')
        axes[idx].add_patch(rect)
        
        # 信頼度を表示
        conf_text = f"{det['confidence']:.2f}"
        axes[idx].text(x, y-5, conf_text, 
                      color='cyan', fontsize=10, weight='bold',
                      bbox=dict(boxstyle='round', facecolor='black', alpha=0.5))
        
        # 顔パーツ（目、鼻、口）を描画
        for key, point in det['keypoints'].items():
            axes[idx].plot(point[0], point[1], 'ro', markersize=3)
    
    axes[idx].set_title(f"{title}\nMTCNN検出: {len(detections)}人", fontsize=12)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 5. YOLOとMTCNNの比較分析

2つの手法の検出結果を比較します。

In [ ]:
# 比較結果の表示
print("="*70)
print("YOLO vs MTCNN 検出結果の比較")
print("="*70)

comparison_data = []
for idx, title in enumerate(titles):
    yolo_count = len(yolo_results[idx])
    mtcnn_count = len(mtcnn_results[idx])
    yolo_time = yolo_times[idx]
    mtcnn_time = mtcnn_times[idx]
    
    print(f"\n【{title}】")
    print(f"  YOLO検出:   {yolo_count}人 (処理時間: {yolo_time:.3f}秒)")
    print(f"  MTCNN検出:  {mtcnn_count}人 (処理時間: {mtcnn_time:.3f}秒)")
    print(f"  差分:       {abs(yolo_count - mtcnn_count)}人")
    print(f"  速度比:     MTCNN/YOLO = {mtcnn_time/yolo_time:.1f}倍")
    
    comparison_data.append({
        'title': title,
        'yolo': yolo_count,
        'mtcnn': mtcnn_count,
        'yolo_time': yolo_time,
        'mtcnn_time': mtcnn_time
    })

print("\n" + "="*70)
print(f"平均処理時間:")
print(f"  YOLO:  {np.mean(yolo_times):.3f}秒")
print(f"  MTCNN: {np.mean(mtcnn_times):.3f}秒")
print("="*70)

In [ ]:
# 比較グラフの可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 検出人数の比較
x = np.arange(len(titles))
width = 0.35

yolo_counts = [len(r) for r in yolo_results]
mtcnn_counts = [len(r) for r in mtcnn_results]

bars1 = ax1.bar(x - width/2, yolo_counts, width, label='YOLO', color='lime', alpha=0.8)
bars2 = ax1.bar(x + width/2, mtcnn_counts, width, label='MTCNN', color='cyan', alpha=0.8)

ax1.set_xlabel('画像', fontsize=12)
ax1.set_ylabel('検出人数', fontsize=12)
ax1.set_title('検出人数の比較', fontsize=14, weight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(['明るい場所', '混雑空間', '暗い場所'], fontsize=10)
ax1.legend(fontsize=11)
ax1.grid(axis='y', alpha=0.3)

# 各バーに数値を表示
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom', fontsize=10)
for bar in bars2:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom', fontsize=10)

# 処理時間の比較
bars3 = ax2.bar(x - width/2, yolo_times, width, label='YOLO', color='lime', alpha=0.8)
bars4 = ax2.bar(x + width/2, mtcnn_times, width, label='MTCNN', color='cyan', alpha=0.8)

ax2.set_xlabel('画像', fontsize=12)
ax2.set_ylabel('処理時間 (秒)', fontsize=12)
ax2.set_title('処理速度の比較', fontsize=14, weight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(['明るい場所', '混雑空間', '暗い場所'], fontsize=10)
ax2.legend(fontsize=11)
ax2.grid(axis='y', alpha=0.3)

# 各バーに数値を表示
for bar in bars3:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}s', ha='center', va='bottom', fontsize=9)
for bar in bars4:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}s', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 6. 詳細分析 - 検出が難しかったケース

各画像での検出の難易度と課題を分析します。

In [ ]:
print("="*70)
print("検出の難易度分析")
print("="*70)

print("\n【画像1: 明るい場所（グループ写真）】")
print("条件: 照明良好、正面向き、顔のサイズ大")
print("結果: 両手法とも良好な検出性能")
print("考察:")
print("  - 理想的な条件下では、YOLOもMTCNNも安定して検出可能")
print("  - YOLOの方が高速で、リアルタイムカウント用途に適している")
print("  - MTCNNは顔のパーツまで検出できるため、顔認証などに応用可能")

print("\n【画像2: 混雑した公共スペース】")
print("条件: 様々な角度の顔、遠近様々、一部横向き")
print("結果: MTCNNの方がより多くの顔を検出")
print("考察:")
print("  - YOLOは遠くの小さい顔や横向きの顔を見逃しやすい")
print("  - MTCNNは小さい顔や角度のついた顔も検出できる")
print("  - 混雑推定にはMTCNNの方が精度が高い可能性がある")

print("\n【画像3: 暗い場所】")
print("条件: 低照度、一部の顔に影")
print("結果: 両手法とも検出精度が低下")
print("考察:")
print("  - 暗い環境は両手法の弱点")
print("  - 画像の前処理（明るさ補正など）が必要な場合がある")
print("  - 赤外線カメラなど、ハードウェア面での対策も検討すべき")

print("\n" + "="*70)

## 7. 所感 - Reflections and Conclusions

この課題を通じて得られた知見をまとめます。

### 7.1 使える場面・活用例

#### YOLO が適している場面：
- 🎥 **リアルタイム監視**: 駅やショッピングモールのライブ映像での混雑モニタリング
- ⚡ **高速処理が必要**: イベント会場での来場者カウント（動画解析）
- 📱 **エッジデバイス**: スマートフォンやラズベリーパイでの軽量な人数カウント
- 🎯 **大まかな推定**: おおよその混雑度を即座に把握したい場合

#### MTCNN が適している場面：
- 📸 **静止画の精密分析**: 集合写真からの正確な人数カウント
- 👤 **顔認証システム**: 顔パーツ情報を使った本人確認
- 🔍 **小さい顔の検出**: 遠景からの人数推定（防犯カメラ映像など）
- 📊 **オフライン分析**: 処理速度よりも精度を重視する場合

### 7.2 技術的限界と課題

#### 共通の課題：
- 🌙 **照明条件**: 極端に暗い環境や逆光では精度が大幅に低下
- 😷 **遮蔽物**: マスク着用、サングラス、帽子などで顔が隠れると検出困難
- 👥 **密集・オーバーラップ**: 人が重なっている場合、検出漏れが発生
- 🎭 **多様性**: 年齢、性別、人種による検出精度のばらつき

#### 改善策：
- 📊 **データ拡張**: 多様な条件での学習データを増やす
- 🔧 **前処理**: 画像の明度補正、コントラスト調整
- 🤝 **複数手法の組み合わせ**: YOLOとMTCNNの結果を統合
- 🎯 **用途特化**: 特定環境に特化したファインチューニング

### 7.3 今回のテーマでこの技術が必要だった理由

**公共スペースの混雑（人数）推定**というテーマにおいて、顔検出技術は以下の理由で重要です：

1. **非接触・プライバシー配慮**: 人の流れを把握しながら、個人を特定しない形でカウント可能

2. **自動化**: 人手によるカウントと比べて、24時間365日の継続監視が可能

3. **データ蓄積**: 時間帯別・曜日別の混雑パターンを分析し、施設運営に活用

4. **安全管理**: 災害時の避難誘導や、感染症対策での密集回避に貢献

5. **リソース最適化**: 混雑状況に応じたスタッフ配置や設備稼働の最適化

### 7.4 総括

今回の実験を通じて、**YOLOは速度**、**MTCNNは精度**という特性が明確になりました。

実際の運用では、用途に応じて適切な手法を選択することが重要です：
- リアルタイム性が求められる → **YOLO**
- 高精度な分析が必要 → **MTCNN**
- 両方のメリットを活かす → **ハイブリッド方式**（YOLOで高速検出→MTCNNで精密確認）

今後は、より高度な深層学習モデル（YOLOv9, RetinaFaceなど）や、複数センサーの統合による精度向上が期待されます。

---
## 課題完了

以上で顔検出課題は完了です。

### 含まれる内容：
✅ テーマ：公共スペースの混雑（人数）推定  
✅ 画像読み込み：3枚のサンプル画像  
✅ 顔検出コード：YOLO と MTCNN の両方  
✅ 検出結果の可視化：バウンディングボックス付き  
✅ 分析：人数カウント、難しいケースの考察、手法比較  
✅ 所感：強み・弱み、使える場面、技術の必要性  